In [12]:
import tensorflow as tf
import SimpleITK as sitk
import numpy as np
import pandas as pd

from tensorflow import keras
from dltk.io import preprocessing
from matplotlib import pyplot as plt

import os

In [13]:
description= pd.read_csv('ADNI_descrip.csv')

In [14]:
description = description.drop(columns= 'Downloaded', axis=1)

In [15]:
description.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,I79080,136_S_1227,MCI,F,65,3,MRI,MPR; ; N3; Scaled,Processed,9/11/2007,NiFTI
1,I66824,136_S_1227,MCI,F,65,2,MRI,MPR; ; N3; Scaled,Processed,2/21/2007,NiFTI
2,I99265,136_S_1227,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,3/06/2008,NiFTI
3,I83554,136_S_0579,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,6/25/2007,NiFTI
4,I119735,136_S_0579,MCI,F,66,2,MRI,MPR-R; ; N3; Scaled_2,Processed,7/10/2006,NiFTI


In [16]:
description = description.loc[1: , :]

In [17]:
description.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
1,I66824,136_S_1227,MCI,F,65,2,MRI,MPR; ; N3; Scaled,Processed,2/21/2007,NiFTI
2,I99265,136_S_1227,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,3/06/2008,NiFTI
3,I83554,136_S_0579,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,6/25/2007,NiFTI
4,I119735,136_S_0579,MCI,F,66,2,MRI,MPR-R; ; N3; Scaled_2,Processed,7/10/2006,NiFTI
5,I82705,136_S_0579,MCI,F,66,3,MRI,MPR; ; N3; Scaled,Processed,1/08/2007,NiFTI


In [18]:
def group_number(group):
    if group == 'MCI':
        result = 1
    elif group == 'AD':
        result = 1
    else:
        result = 0
    return result




description['group_number'] = description['Group'].apply(lambda x: group_number(x))


In [19]:
description.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,group_number
1,I66824,136_S_1227,MCI,F,65,2,MRI,MPR; ; N3; Scaled,Processed,2/21/2007,NiFTI,1
2,I99265,136_S_1227,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,3/06/2008,NiFTI,1
3,I83554,136_S_0579,MCI,F,66,4,MRI,MPR; ; N3; Scaled,Processed,6/25/2007,NiFTI,1
4,I119735,136_S_0579,MCI,F,66,2,MRI,MPR-R; ; N3; Scaled_2,Processed,7/10/2006,NiFTI,1
5,I82705,136_S_0579,MCI,F,66,3,MRI,MPR; ; N3; Scaled,Processed,1/08/2007,NiFTI,1


In [20]:
import SimpleITK as sitk
import nibabel as nib

In [21]:
ROOt = './data-folder/'
images = 'I101432.nii'

In [22]:
first = nib.load(ROOt + images)

In [24]:
image_data = first.get_fdata()
image_data

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [25]:
def load_nii(id):
    path = './data-folder/' + id +'.nii';
    t1_img = nib.load(path);
    return t1_img;

description['images'] = description['Image Data ID'].apply(load_nii)


In [248]:
import os 

len(os.listdir('data-folder/'))

419

In [238]:
_ = load_nii(description['Image Data ID'][1])

In [244]:
len(images)

11

In [253]:
import keras.backend as K
from keras.preprocessing.image import *
import nibabel as nib
import os
import numpy as np
import threading
import warnings
import multiprocessing.pool
from functools import partial

from PIL import Image as pil_image 
from imageio import imwrite
# from scipy.misc import imsave


backend = K

from matplotlib import pyplot as plt

def show_image(img_data):
    plt.imshow(img_data.T, interpolation='nearest', cmap="gray")
    plt.show()
    
import numpy as np

def pad_image(img_data, target_dims):
    left_pad = round(float(target_dims[0] - img_data.shape[0]) / 2)
    right_pad = round(float(target_dims[0] - img_data.shape[0]) - left_pad)
    top_pad = round(float(target_dims[1] - img_data.shape[1]) / 2)
    bottom_pad = round(float(target_dims[1] - img_data.shape[1]) - top_pad)
    front_pad = round(float(target_dims[2] - img_data.shape[2]) / 2)
    back_pad = round(float(target_dims[2] - img_data.shape[2]) - front_pad)

    pads = ((left_pad, right_pad),
            (top_pad, bottom_pad),
            (front_pad, back_pad))

    new_img = np.zeros((target_dims))
    new_img[:,:,:] = np.pad(img_data[:,:,:], pads, 'constant', constant_values=0)

    return new_img

class NIfTIDirectoryIterator(Iterator):
    def __init__(self, directory, image_data_generator,
                 augmentations=None,
                 target_size=(256, 256, 256), num_channels=1,
                 num_patches=1,
                 classes=None, class_mode='categorical',
                 batch_size=32, shuffle=True, seed=None,
                 save_to_dir=None, save_prefix='', save_format='png',
                 axial_slice=None,
                 follow_links=False, split=None):
        self.directory = directory
        self.image_data_generator = image_data_generator
        self.augmentations = augmentations
        self.target_size = tuple(target_size)
        self.num_channels = num_channels
        self.num_patches = num_patches
        self.image_shape = self.target_size + (num_channels,)
        self.classes = classes
        if class_mode not in {'categorical', 'binary', 'sparse',
                              'input', None}:
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", "input"'
                             ' or None.')
        self.class_mode = class_mode

        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        self.axial_slice = axial_slice

        white_list_formats = {'nii', 'nii.gz'}

        # Counter number of samples and classes
        self.samples = 0

        if not classes:
            classes = []
            for subdir in sorted(os.listdir(directory)):
                if os.path.isdir(os.path.join(directory, subdir)):
                    classes.append(subdir)
        self.num_classes = len(classes)
        self.class_indices = dict(zip(classes, range(len(classes))))

        pool = multiprocessing.pool.ThreadPool()

        function_partial = partial(_count_valid_files_in_directory,
                                   white_list_formats=white_list_formats,
                                   follow_links=follow_links,
                                   split=None)
        self.samples = sum(pool.map(function_partial,
                                    (os.path.join(directory, subdir)
                                     for subdir in classes)))
        print('Found %d images belonging to %d classes.' %
              (self.samples, self.num_classes))

        # Build an index of the images in the different class subfolders
        results = []
        self.filenames = []
        self.classes = np.zeros((self.samples,), dtype='int32')
        i = 0
        for dirpath in (os.path.join(directory, subdir) for subdir in classes):
            results.append(
                pool.apply_async(_list_valid_filenames_in_directory,
                                 (dirpath, white_list_formats, split,
                                  self.class_indices, follow_links)))
        for res in results:
            classes, filenames = res.get()
            self.classes[i:i + len(classes)] = classes
            self.filenames += filenames
            i += len(classes)

        pool.close()
        pool.join()

        super(NIfTIDirectoryIterator, self).__init__(self.samples,
                                                     batch_size,
                                                     shuffle,
                                                     seed)

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros(
            (len(index_array) * self.num_patches,) + self.image_shape,
            dtype=backend.floatx())
        # build batch of image data
        i = 0
        for j in index_array:
            fname = self.filenames[j]
            img = nib.load(os.path.join(self.directory, fname))
            # TODO: pad/crop image to target_size here
            x = img.get_data()
            #x = pad_image(x, self.target_size)
            x = pad_image(x, (256,256,256))
            # TODO: extensible image augmentation applied here
            if self.augmentations:
                for aug, params in self.augmentations.items():
                    x = aug(x, **params)

            # TODO: figure out how to handle multi-chan data; for now 3D volumes with 1 chan
            if len(batch_x.shape)-1 == len(x.shape):
                x = np.reshape(x, x.shape + (self.num_channels,))
                batch_x[i] = x
                i += 1
            else: # handle multi patches
                for transformed_img in x:
                    batch_x[i] = transformed_img
                    i += 1


        if self.save_to_dir:
            for i, j in enumerate(index_array):

                # if 3D training images, save a single slice to disk
                if len(batch_x.shape) == 5:
                    if self.axial_slice:
                        img = batch_x[i,:,:,self.axial_slice,0].T
                    else:
                        middle_slice = batch_x.shape[3]//2
                        img = batch_x[i,:,:,middle_slice,0].T
                    fname = '{prefix}_{index}_{hash}.{format}'.format(
                            prefix=self.save_prefix,
                            index=j,
                            hash=np.random.randint(1e7),
                            format=self.save_format)
                    fname = os.path.join(self.save_to_dir, fname)
                    imsave(fname, img)
                # if 2D training images, save that 2D image to disk
                elif len(batch_x.shape) == 4: 
                    img = batch_x[i,:,:,0].T
                    fname = '{prefix}_{index}_{hash}.{format}'.format(
                            prefix=self.save_prefix,
                            index=j,
                            hash=np.random.randint(1e7),
                            format=self.save_format)
                    fname = os.path.join(self.save_to_dir, fname)
                    imsave(fname, img)

        # build batch of labels
        if self.class_mode == 'input':
            batch_y = batch_x.copy()
        elif self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(backend.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros(
                (len(batch_x), self.num_classes),
                dtype=backend.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x
        return batch_x, batch_y

    def next(self):
        with self.lock:
            index_array = next(self.index_generator)
        return self._get_batches_of_transformed_samples(index_array)


class NIfTIImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory,
                            augmentations=None,
                            target_size=(256, 256, 256), num_channels=1,
                            num_patches=1,
                            classes=None, class_mode='categorical',
                            batch_size=32, shuffle=True, seed=None,
                            save_to_dir=None, save_prefix='', save_format='png',
                            axial_slice=None,
                            follow_links=False):
        return NIfTIDirectoryIterator(directory, self,
                                      augmentations=augmentations,
                                      target_size=target_size, num_channels=num_channels,
                                      num_patches=num_patches,
                                      classes=classes, class_mode=class_mode,
                                      batch_size=batch_size, shuffle=shuffle, seed=seed,
                                      save_to_dir=save_to_dir, save_prefix=save_prefix, 
                                      save_format=save_format,
                                      axial_slice=axial_slice,
                                      follow_links=follow_links)

    def apply_transform(self, x, transform_parameters):
        pass

    def get_random_transform(self, img_shape, seed=None):
        pass

    def standardize(self, x):
        pass



class NIfTINumpyArrayIterator(NumpyArrayIterator):
    # TODO: change some stuff to work with nibabel
    def __init__(self):
        raise NotImplementedError


def _iter_valid_files(directory, white_list_formats, follow_links):
    """Iterates on files with extension in `white_list_formats` contained in `directory`.
   # Arguments
    directory: Absolute path to the directory
    containing files to be counted
    white_list_formats: Set of strings containing allowed extensions for
    the files to be counted.
    follow_links: Boolean.
   # Yields
    Tuple of (root, filename) with extension in `white_list_formats`.
    """
    def _recursive_list(subpath):
        return sorted(os.walk(subpath, followlinks=follow_links),
                      key=lambda x: x[0])

    for root, _, files in _recursive_list(directory):
        for fname in sorted(files):
            for extension in white_list_formats:
                if fname.lower().endswith('.tiff'):
                    warnings.warn('Using \'.tiff\' files with multiple bands '
                                  'will cause distortion. '
                                  'Please verify your output.')
                if fname.lower().endswith('.' + extension):
                    yield root, fname

def _count_valid_files_in_directory(directory,
                                    white_list_formats,
                                    split,
                                    follow_links):
    ''' has to be implemented again just because of how partial works'''
    num_files = len(list(
        _iter_valid_files(directory, white_list_formats, follow_links)))
    if split:
        start, stop = int(split[0] * num_files), int(split[1] * num_files)
    else:
        start, stop = 0, num_files
    return stop - start


def _list_valid_filenames_in_directory(directory, white_list_formats, split,
                                       class_indices, follow_links):
    """Lists paths of files in `subdir` with extensions in `white_list_formats`.
    # Arguments
    directory: absolute path to a directory containing the files to list.
    The directory name is used as class label
    and must be a key of `class_indices`.
    white_list_formats: set of strings containing allowed extensions for
    the files to be counted.
    split: tuple of floats (e.g. `(0.2, 0.6)`) to only take into
    account a certain fraction of files in each directory.
    E.g.: `segment=(0.6, 1.0)` would only account for last 40 percent
    of images in each directory.
    class_indices: dictionary mapping a class name to its index.
    follow_links: boolean.
   # Returns
   classes: a list of class indices
   filenames: the path of valid files in `directory`, relative from
   `directory`'s parent (e.g., if `directory` is "dataset/class1",
   the filenames will be
    `["class1/file1.jpg", "class1/file2.jpg", ...]`).
    """
    dirname = os.path.basename(directory)
    if split:
        num_files = len(list(
            _iter_valid_files(directory, white_list_formats, follow_links)))
        start, stop = int(split[0] * num_files), int(split[1] * num_files)
        valid_files = list(
            _iter_valid_files(
                directory, white_list_formats, follow_links))[start: stop]
    else:
        valid_files = _iter_valid_files(
            directory, white_list_formats, follow_links)

    classes = []
    filenames = []
    for root, fname in valid_files:
        classes.append(class_indices[dirname])
        absolute_path = os.path.join(root, fname)
        relative_path = os.path.join(
            dirname, os.path.relpath(absolute_path, directory))
        filenames.append(relative_path)

    return classes, filenames

In [ ]:
params = {
    # 'target_size': (256, 256, 256),
    'target_size': patch_size,
    'batch_size': batch_size,
    'class_mode': 'categorical',
    'num_patches': num_patches,
    # 'axial_slice': 2,
    #'save_to_dir': SAMPLE_AUG_PATH,
    #'save_prefix': AUG_FILE_PREFIX,
}

In [286]:
import tensorflow as tf

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def load_nii(self, id):
        path = './data-folder/' + id +'.nii';
        t1_img = nib.load(path);
        return t1_img;   
    
    def daxis(self, images):
        image_data = images.get_fdata()
        x_slice = image_data[119, :, :]
        return x_slice

            
    def __get_data(self, batches):
        images_ = []

        for id_ in batches['Image Data ID']:
            image_ = self.load_nii( id_)
            image_ = daxis(image_)
            images_.append(image_)

        data_ = np.vstack(images_)
        
        return data_, batches.group_number.to_numpy()
        
    
    def __getitem__(self, index):
        
        batches = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [262]:
%config Completer.use_jedi = False

In [287]:
generator = CustomDataGen(description, batch_size=1)


In [294]:
generator.__getitem__(415)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0], dtype=int64))

Image Data ID    0
Subject          0
Group            0
Sex              0
Age              0
Visit            0
Modality         0
Description      0
Type             0
Acq Date         0
Format           0
group_number     0
images           0
axis             0
dtype: int64

In [267]:
gen = tf.keras.preprocessing.image.ImageDataGenerator()
gen.flow_from_dataframe()

In [273]:
np.vstack(description.head(2).axis[1])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 170 and the array at index 2 has size 256

In [242]:
images_ = []

for id_ in description['Image Data ID']:
    image_ = load_nii(id_)
    image_ = daxis(image_)
    images_.append(image_)
    
data_ = np.vstack(images_)

MemoryError: Unable to allocate 75.0 MiB for an array with shape (240, 256, 160) and data type float64

In [196]:
def daxis (images):
    image_data = images.get_fdata()
    x_slice = image_data[119, :, :]
    return x_slice
    x_slice = np.pad(x_slice, (0, int((256 - x_slice.shape[1])/2)))
    y_slice = image_data[:, 127, :]
    y_slice = np.pad(y_slice, (0, int((256 - y_slice.shape[1])/2)))
    z_slice = image_data[:, :, 79]
    slices = np.array([x_slice,y_slice,z_slice])
    return slices

In [165]:
description['axis2'] = description['images'].apply(lambda x: daxis(x))

TypeError: `pad_width` must be of integral type.

In [215]:
s = daxis(description.iloc[0]['images'])
s.shape

(170,)

In [223]:
np.pad(s, (0,int((256 - s.shape[0])/2))).shape

(256, 170)

In [19]:
description['group_number']

1      1
2      1
3      1
4      1
5      1
      ..
415    0
416    0
417    0
418    0
419    0
Name: group_number, Length: 419, dtype: int64

In [20]:
description['axis']

1      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
2      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
3      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
4      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
5      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
                             ...                        
415    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
416    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
417    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
418    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
419    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
Name: axis, Length: 419, dtype: object

creating two lists full of arrays so that I can feed them into the model. Our y variable will be the '1s' where the image shows some indication of Alzheimers, or '2s' where the image is normal.

Trying the .loc method to grab the specific arrays that have at least a mild cognitive impairment (MCI) diagnosis and turning them into a list to pass into the model.

In [21]:
alzheimers= description.loc[description['group_number'] == 1]

In [22]:
alzheimers['axis']

1      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
2      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
3      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
4      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
5      [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
                             ...                        
407    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
408    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
409    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
410    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
411    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
Name: axis, Length: 283, dtype: object

In [23]:
x = alzheimers['axis'].tolist()

In [24]:
no_alzheimers= description.loc[description['group_number']== 0]

In [25]:
no_alzheimers

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,group_number,images,axis
19,I40269,136_S_0196,CN,F,78,2,MRI,MPR; ; N3; Scaled,Processed,5/01/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
20,I66740,136_S_0196,CN,F,79,4,MRI,MPR; ; N3; Scaled,Processed,5/07/2007,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
21,I119725,136_S_0196,CN,F,78,2,MRI,MPR; ; N3; Scaled_2,Processed,5/01/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
22,I40281,136_S_0196,CN,F,79,3,MRI,MPR; ; N3; Scaled,Processed,10/23/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
32,I119714,136_S_0184,CN,F,79,3,MRI,MPR; ; N3; Scaled_2,Processed,9/06/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,I45126,002_S_0559,CN,M,79,2,MRI,MPR; ; N3; Scaled,Processed,6/27/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
416,I40657,002_S_0413,CN,F,76,2,MRI,MPR; ; N3; Scaled,Processed,5/19/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
417,I128346,002_S_0413,CN,F,77,3,MRI,MPR; ; N3; Scaled_2,Processed,11/15/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
418,I118675,002_S_0413,CN,F,76,2,MRI,MPR; ; N3; Scaled_2,Processed,5/19/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [26]:
image_list = no_alzheimers['axis'].tolist()

In [27]:
image_list

[[array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         ...,
         [ 16.41026115,  32.65826416,  14.62392139, ...,  21.49555016,
           36.73676682,  40.61392593],
         [ 60.76596069, 138.90927124, 169.1257782 , ...,  32.89064407,
           35.4854393 ,  31.74275208],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0

In [28]:
%store image_list
%store alzheimers_list

Stored 'image_list' (list)
Stored 'alzheimers_list' (list)


Just setting up for when we do the modeling

In [235]:
description.axis.iloc[1][2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [230]:
X= description['axis'].values.astype(np.float32)
y= description['group_number'].values.astype(np.float32)

ValueError: setting an array element with a sequence.

In [163]:
X_train[1][0].shape

(256, 160)

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)


In [114]:
y

1      1
2      1
3      1
4      1
5      1
      ..
415    0
416    0
417    0
418    0
419    0
Name: group_number, Length: 419, dtype: int64

In [302]:
description.sample(int(.75 * description.shape[0]))

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,group_number,images,axis
195,I65374,027_S_1385,AD,F,70,2,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/26/2007,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
21,I119725,136_S_0196,CN,F,78,2,MRI,MPR; ; N3; Scaled_2,Processed,5/01/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
240,I88270,023_S_1046,MCI,M,73,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,11/06/2007,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
220,I77044,023_S_1262,AD,F,73,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,9/04/2007,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
137,I89124,067_S_0290,MCI,M,75,2,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,6/01/2006,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,I94563,051_S_1123,CN,F,77,4,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,2/18/2008,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
71,I91113,127_S_0622,CN,F,75,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,12/13/2006,NiFTI,0,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,I82705,136_S_0579,MCI,F,66,3,MRI,MPR; ; N3; Scaled,Processed,1/08/2007,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
213,I87547,027_S_0404,AD,F,89,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,5/16/2007,NiFTI,1,<class 'nibabel.nifti1.Nifti1Image'>\ndata sha...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [108]:
y.shape

(419,)

In [ ]:
X = X.reshape(())
y= y.reshape(())

In [117]:
epochs = 10
batch_size = 10
input_shape = (300,300,3)

In [118]:
from tensorflow.keras.layers import Dense, Input

In [119]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from tensorflow.keras import datasets, layers, models

In [277]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['recall'])

In [225]:
np.matrix(X_train)

matrix([[list([array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.63409662, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
 

In [298]:
history = model.fit_generator(generator, epochs=epochs, verbose=1)

C:\Users\Kamryn\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: Failed to find data adapter that can handle input: <class '__main__.CustomDataGen'>, <class 'NoneType'>